In [1]:
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
#%matplotlib inline

In [11]:
class CosmoSim:
    
    Npts = -1 # Number of points per side of square
    ptPosArray = np.empty(1)
    ptVelArray = np.empty(1)
    ptMass = -1.0
    G = 6.67408E-11 # m^3 / (kg * s^2)
    sideLength = -1.0
    t = 0.0
    outdir = ''
    force_matrix = None
    
    def __init__(self,Npts, ptMass, sideLength, t, outdir):
        
        self.outdir = outdir
        self.Npts = Npts
        
        self.ptPosArray = []
        for i in range(Npts):
            for j in range(Npts):
                self.ptPosArray.append([i,j])
        self.ptPosArray = np.array(self.ptPosArray)
        
        self.ptVelArray = []
        for i in range(Npts):
            for j in range(Npts):
                self.ptVelArray.append([0,0])
        self.ptVelArray = np.array(self.ptVelArray)
        
        self.ptMass = ptMass
        self.sideLength = sideLength
        self.t = t
        return
    
    def perturbPos(self, strength):
        theta = np.random.uniform(0.0,2*np.pi,self.Npts**2)
        perturb = strength*np.array([[np.cos(th),np.sin(th)] for th in theta])
        self.ptPosArray = self.ptPosArray + perturb
    
    def periodicDistance(self,x1, y1, x2, y2, L):
        abs_dx = np.abs(x1-x2)
        abs_dy = np.abs(y1-y2)
        xsep = min([abs_dx,L-abs_dx])
        ysep = min([abs_dy, L-abs_dy])
        return np.sqrt(xsep**2 + ysep**2)
    
    def evolve(self, tfinal, N, padding=1.0, showVel=False, snapshot=False):
        self.dt = float(tfinal)/N
        count=0
        while self.t < tfinal:
            self.verletUpdateStep()
            if snapshot:
                f,ax = self.plotSim(padding, showVel)
                f.savefig(self.outdir+'snap_%08d.pdf'%(count))
            self.t = self.t + self.dt
            count += 1
        return
    
    
    def verletUpdateStep(self):
        if self.force_matrix is None:
            self.force_matrix = self.calculateForces()
        vel_half = self.ptVelArray + 0.5*self.dt*np.sum(self.force_matrix, axis=1)
        self.ptPosArray = self.ptPosArray + self.dt*vel_half
        self.ptPosArray = self.ptPosArray%self.sideLength
        self.force_matrix = self.calculateForces()
        self.ptVelArray = vel_half + 0.5*self.dt*np.sum(self.force_matrix, axis=1)
        return
    
    def calculateForces(self):
        force_matrix = np.ndarray((self.Npts**2,self.Npts**2,2))
        for i in range(self.Npts**2):
            force_matrix[i,i] = np.array([0,0])
        for i in range(self.Npts**2):
            for j in range(i+1,self.Npts**2):
                xi, yi = self.ptPosArray[i]
                xj, yj = self.ptPosArray[j]
                distance = self.periodicDistance(xi, yi, xj, yj, self.sideLength)
                force_direction = 1.0/distance*np.array([xj - xi, yj - yi])
                force_matrix[i,j] = 1.0 / distance**2 * force_direction
                force_matrix[j,i] = np.copy(force_matrix[i,j])
        return force_matrix
        
        
        
    def plotSim(self,padding,showVel=False):
        f,ax = plt.subplots()
        ax.scatter(self.ptPosArray[:,0],self.ptPosArray[:,1],marker='.',color='k')
        if showVel:
            ax.quiver(self.ptPosArray[:,0], self.ptPosArray[:,1], \
                   self.ptVelArray[:,0],self.ptVelArray[:,1],angles='xy',scale=50.0,color='r',headwidth=2)
        
        ax.set_xlim((-padding,self.sideLength))
        ax.set_ylim((-padding,self.sideLength))
        ax.set_title('$t=$%f'%(self.t))
        
        ax.tick_params(
        axis='x',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        bottom='off',      # ticks along the bottom edge are off
        top='off',         # ticks along the top edge are off
        labelbottom='off') # labels along the bottom edge are off
        ax.tick_params(
        axis='y',          # changes apply to the x-axis
        which='both',      # both major and minor ticks are affected
        left='off',      # ticks along the bottom edge are off
        right='off',         # ticks along the top edge are off
        labelleft='off') # labels along the bottom edge are off
        
        return f,ax
        

In [3]:
sim = CosmoSim(10, 1, 10,0.0,'.')

In [4]:
f,ax = sim.plotSim(1,showVel=False)

In [5]:
sim.perturbPos(.2)

In [6]:
f,ax = sim.plotSim(1,showVel=False)

In [13]:
sim = CosmoSim(Npts=10, ptMass=1, sideLength=10,t=0.0,outdir='/home/data/mew488/Desktop/NYU/Research/Hogg/reconstruct/sim1/')
sim.perturbPos(.2)
sim.evolve(tfinal=1.0, N=100, padding=1.0, showVel=False, snapshot=True)

/home/data/mew488/virtualenvs/mypy2.7/lib/python2.7/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [110]:
a

array([0, 1, 2, 3, 4])

In [113]:
a = np.reshape(np.arange(10),(5,2))

In [7]:
a=1

In [10]:
print "snap_%08d"%(a)
print "snap_%08d"%(100)

snap_00000001
snap_00000100
